# Codegen Tutorial

One of the most important features of symforce is the ability to generate computationally efficient code from symbolic expressions. Before progressing, first make sure you are familiar with the other symforce tutorials, especially the [Values tutorial](../notebooks/values_tutorial.html).

The typical workflow for generating a function is as follows:

1. Build an input Values object that defines a symbolic representation of each input to the function. Note that inputs and outputs can be Values objects themselves, which symforce will automatically generate into custom types.
2. Build an output Values object that defines the outputs of the function in terms of the objects in the input Values.
3. Generate the code in your desired language

Aternately, symforce can automatically generate a function given a python function and a list of the types of the inputs.

In [ ]:
# Setup
import os
import symforce
symforce.set_backend('symengine')
symforce.set_log_level('warning')

from symforce import geo
from symforce import ops
from symforce import sympy as sm
from symforce import types as T
from symforce.values import Values
from symforce.codegen import CodegenMode
from symforce.codegen import codegen_util
from symforce.codegen import Codegen
from symforce.notebook_util import display, display_code, display_code_file

## Code generation using implicit functions

First we look at generating functions using a list of input variables and output expressions that are a function of those variables. In this case we don't need to explicitly define a function in python, but can instead generate one directly using the codegen package.

Let's set up an example for the [double pendulum](https://www.myphysicslab.com/pendulum/double-pendulum-en.html). We'll skip the derivation and just define the equations of motion for the angular acceleration of the two links:

In [ ]:
# Define symbols
L = geo.V2.symbolic('L').T      # Length of the two links
m = geo.V2.symbolic('m').T      # Mass of the two links
ang = geo.V2.symbolic('a').T    # Angle of the two links
dang = geo.V2.symbolic('da').T  # Angular velocity of the two links
g = sm.Symbol('g')              # Gravity

In [ ]:
# Angular acceleration of the first link
ddang_0 = (
    -g * (2 * m[0] + m[1]) * sm.sin(ang[0])
    - m[1] * g * sm.sin(ang[0] - 2 * ang[1])
    - 2
    * sm.sin(ang[0] - ang[1])
    * m[1]
    * (dang[1] * 2 * L[1] + dang[0] * 2 * L[0] * sm.cos(ang[0] - ang[1])))\
    / (L[0] * (2 * m[0] + m[1] - m[1] * sm.cos(2 * ang[0] - 2 * ang[1])))
display(ddang_0)

In [ ]:
# Angular acceleration of the second link
ddang_1 = (
    2
    * sm.sin(ang[0] - ang[1])
    * (
        dang[0] ** 2 * L[0] * (m[0] + m[1])
        + g * (m[0] + m[1]) * sm.cos(ang[0])
        + dang[1] ** 2 * L[1] * m[1] * sm.cos(ang[0] - ang[1])
    )
) / (L[1] * (2 * m[0] + m[1] - m[1] * sm.cos(2 * ang[0] - 2 * ang[1])))
display(ddang_1)

Now let's organize the input symbols into a Values hierarchy:

In [ ]:
inputs = Values()

inputs['ang'] = ang
inputs['dang'] = dang
    
with inputs.scope('constants'):
    inputs['g'] = g

with inputs.scope('params'):
    inputs['L'] = L
    inputs['m'] = m

display(inputs)

The output will simply be a 2-vector of the angular accelerations:

In [ ]:
outputs = Values(
    ddang=geo.V2(ddang_0, ddang_1)
)

display(outputs)

Now run code generation to produce an executable module (in a temp directory if none provided):

In [ ]:
double_pendulum = Codegen(
    inputs=inputs,
    outputs=outputs,
    mode=CodegenMode.CPP,
    name='double_pendulum',
    return_key='ddang'
)
double_pendulum_data = double_pendulum.generate_function()

# Print what we generated
print('Files generated in {}:\n'.format(double_pendulum_data['output_dir']))
for f in double_pendulum_data['generated_files']:
    print('  |- {}'.format(os.path.relpath(f, double_pendulum_data['output_dir'])))

In [ ]:
display_code_file(os.path.join(double_pendulum_data['cpp_function_dir'], 'double_pendulum.h'), 'C++')

We can also generate functions with different function declarations:

In [ ]:
# Function using structs as inputs and outputs (returned as pointer arg)
input_values = Values(inputs=inputs)
output_values = Values(outputs=outputs)
namespace = 'double_pendulum'
double_pendulum_values  = Codegen(
    inputs=input_values,
    outputs=output_values,
    mode=CodegenMode.CPP,
    name='double_pendulum'
)
double_pendulum_values_data = double_pendulum_values.generate_function(namespace=namespace)

# Print what we generated. Note the nested structs that were automatically generated.
print('Files generated in {}:\n'.format(double_pendulum_values_data['output_dir']))
for f in double_pendulum_values_data['generated_files']:
    print('  |- {}'.format(os.path.relpath(f, double_pendulum_values_data['output_dir'])))

display_code_file(os.path.join(double_pendulum_values_data['cpp_function_dir'], 'double_pendulum.h'), 'C++')

Finally, we can generate the same function in other languages as well:

In [ ]:
namespace = 'double_pendulum'
double_pendulum_python = Codegen(
    inputs=inputs,
    outputs=outputs,
    mode=CodegenMode.PYTHON2,
    name='double_pendulum',
    return_key='ddang'
)
double_pendulum_python_data = double_pendulum_python.generate_function(namespace=namespace)

print('Files generated in {}:\n'.format(double_pendulum_python_data['output_dir']))
for f in double_pendulum_python_data['generated_files']:
    print('  |- {}'.format(os.path.relpath(f, double_pendulum_python_data['output_dir'])))

display_code_file(os.path.join(double_pendulum_python_data['python_function_dir'], 'double_pendulum.py'), 'python')

In [ ]:
types_module = codegen_util.load_generated_package(os.path.join(double_pendulum_python_data["python_types_dir"], namespace))

ang = [0.0, 0.5]
dang = [0.0, 0.0]
consts = types_module.constants_t()
consts.g = 9.81
params = types_module.params_t()
params.L = [0.5, 0.3]
params.m = [0.3, 0.2]

gen_module = codegen_util.load_generated_package(double_pendulum_python_data["python_function_dir"])
gen_module.double_pendulum(ang, dang, consts, params)

## Generating explicit functions

Next we look at using existing python functions to generate an equivalent function using the codegen package. Here we must specify the inputs of the function, and can decide how the generated function is declared (e.g. whether to return an object using a return statement or a pointer passed as an argument to the function).

In [ ]:
def az_el_from_point(nav_T_cam: geo.Pose3, nav_t_point: geo.Vector3, epsilon: T.Scalar=0) -> geo.Vector2:
    """
    Transform a nav point into azimuth / elevation angles in the
    camera frame.

    Args:
        nav_T_cam (geo.Pose3): camera pose in the world
        nav_t_point (geo.Matrix): nav point
        epsilon (Scalar): small number to avoid singularities

    Returns:
        geo.Matrix: (azimuth, elevation)
    """
    cam_t_point = nav_T_cam.inverse() * nav_t_point
    x, y, z = cam_t_point
    theta = sm.atan2(y, x + epsilon)
    phi = sm.pi / 2 - sm.acos(z / (cam_t_point.norm() + epsilon))
    return geo.V2(theta, phi)

In [ ]:
az_el_codegen = Codegen.function(
    name='AzElFromPoint',
    func=az_el_from_point,
    mode=CodegenMode.CPP
)
az_el_codegen_data = az_el_codegen.generate_function()

print('Files generated in {}:\n'.format(az_el_codegen_data['output_dir']))
for f in az_el_codegen_data['generated_files']:
    print('  |- {}'.format(os.path.relpath(f, az_el_codegen_data['output_dir'])))

display_code_file(os.path.join(az_el_codegen_data['generated_files'][0]), 'C++')

## Generating functions jacobians

In [ ]:
codegen_with_jacobians = az_el_codegen.create_with_derivatives(
    # Just compute wrt the pose and point, not epsilon
    which_args=[0, 1],
    # Include value, not just jacobians
    include_result=True
)

data = codegen_with_jacobians.generate_function()
from symforce.notebook_util import display_code_file
display_code_file(os.path.join(data['generated_files'][0]), 'C++')